In [3]:
import cv2
import os
import pickle
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt 


# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #base
        self.base = None
        # real world parameters
        self.rw_xfitted = []
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in pixels of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None  

In [33]:
# Creating objects for left and right lines
left_line = Line()
right_line = Line()

# Reading the saved camera matrix and distortion coefficients
cam_pickle = pickle.load( open( "camera_parameters.pickle", "rb") ) 
mtx = cam_pickle[0]
dist = cam_pickle[1]

# Reading perspective transformation
M_pickle = pickle.load( open( "p_transformation.pickle", "rb") ) 
M = M_pickle[0]
Minv = M_pickle[1]

def extract_lanes (img, lightness_th = 210, hue_th = (15, 35), sobelx_th=(30, 100)): 
    
    # Convert to HLS color space and separate the S and H channel 
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    l_channel = hls[:,:,1]
    h_channel = hls[:,:,0]
    s_channel = hls[:,:,2]
    
    # white color has a very high saturation
    binary_l = np.zeros_like(l_channel)
    binary_l[(l_channel >= lightness_th)] = 1
    
    # yellow color has a H between 20 and 30 
    binary_h = np.zeros_like(h_channel)
    binary_h[(h_channel <= hue_th[1]) & (h_channel >= hue_th[0])] = 1   #30-20
           
    # checking for gradient in x and y in the saturation channel  
    sobel_x = cv2.Sobel(s_channel, cv2.CV_64F, 1, 0, 5) # x direction
    sobel_y = cv2.Sobel(s_channel, cv2.CV_64F, 0, 1, 5) # y direction 
    
    # scalating the gradients
    abs_sobel_x = np.absolute(sobel_x)
    scaled_sobel_x = np.uint8(255*abs_sobel_x/np.max(abs_sobel_x))   
    abs_sobel_y = np.absolute(sobel_y)
    scaled_sobel_y = np.uint8(255*abs_sobel_y/np.max(abs_sobel_y))
    
    # applying threshold in x direction 
    binary_x = np.zeros_like(scaled_sobel_x)
    binary_x[(scaled_sobel_x > sobelx_th[0]) & (scaled_sobel_x < sobelx_th[1])] = 1
        
    # Combine the three binary thresholds
    combined_binary = np.zeros_like(binary_l)
    combined_binary[(binary_l == 1) | (binary_h == 1) | (binary_x ==1)] = 1
       
    return combined_binary


def find_lane_pixels(binary_warped):
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    
    #ask if any of the lines hasn't been detected to look for the base points again
    if not (left_line.detected and right_line.detected):
    # Take a histogram of the bottom half of the image
        histogram = np.sum(binary_warped[binary_warped.shape[0]//3:,:], axis=0)        
        # Find the peak of the left and right halves of the histogram
        # These will be the starting point for the left and right lines
        midpoint = np.int(histogram.shape[0]//2)
        leftx_base = np.argmax(histogram[:midpoint])
        rightx_base = np.argmax(histogram[midpoint:]) + midpoint
            
    else : 
        leftx_base = np.int(left_line.base) 
        rightx_base = np.int(right_line.base) 
        

    if (leftx_base < 200.0 or leftx_base > 400.0 or rightx_base < 800 or rightx_base > 1170 ):
            leftx_base = 300 
            rightx_base = 1000 
    # HYPERPARAMETERS
    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0,255,0), 2) 
        
        # Identify the nonzero pixels in x and y within the window #
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        # If you found > minpix pixels, recenter next window on their mean position
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:        
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    print()

    return leftx, lefty, rightx, righty, out_img


def fit_polynomial(binary_warped):
    
    # Find our lane pixels first
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)

    ym_per_pix = 35/700 # meters per pixel in y dimension
    xm_per_pix = 3.7/660 # meters per pixel in x dimension
    
   
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:      
        # Fit a second order polynomial to each using `np.polyfit`
        left_fit = np.polyfit(lefty, leftx, 2)
        right_fit = np.polyfit(righty, rightx, 2)       
        left_fit_rw = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
        right_fit_rw = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)   
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
        left_line.detected = True
        right_line.detected = True
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        # If there's an error, retrieve the information from a previously detected line
        left_fitx = left_line.recent_xfitted
        right_fitx = right_line.recent_xfitted
        left_fit = left_line.fit
        right_fit = right_line.fit       
        left_fit_rw = left_line.rw_xfitted
        right_fit_rw = right_line.rw_xfitted
        left_line.detected = False
        right_line.detected = False
        
    #saving x values of the last n fits of the line
    left_line.allx =  left_fitx.copy() 
    right_line.ally = right_fitx.copy()
    
    # measuring the curvature
    y_eval = np.max(ploty)*ym_per_pix
    left_curverad = ((1 + (2*left_fit_rw[0]*y_eval+left_fit_rw[1])**2)**(3/2))/(2*left_fit_rw[0])  
    right_curverad = ((1 + (2*right_fit_rw[0]*y_eval+right_fit_rw[1])**2)**(3/2))/(2*right_fit_rw[0])    
    left_line.radius_of_curvature =  left_curverad 
    right_line.radius_of_curvature = right_curverad
    
    # measuring how centered the vehicle is 
    mploty = np.max(ploty)
    left_x = left_fit[0]*mploty**2 + left_fit[1]*mploty + left_fit[2]
    right_x = right_fit[0]*mploty**2 + right_fit[1]*mploty + right_fit[2]
    deviation = (640 - (0.5 * left_x + 0.5 * right_x)) * xm_per_pix 
    
    #saving distance in pixels of vehicle center from the line and checking for starting position in the image
    left_line.line_base_pos =  (640 - left_x)  
    right_line.line_base_pos = (right_x - 640)
    
    if (left_line.line_base_pos < 200.0 or left_line.line_base_pos > 500.0 ):
        left_line.detected = False      
        
    if (right_line.line_base_pos < 200.0 or right_line.line_base_pos > 500.0 ):
        right_line.detected = False  
     
    # good fit, I'm defining a good fit 
    if (left_line.detected and right_line.detected):
        left_line.base = left_x
        right_line.base = right_x
        left_line.recent_xfitted = left_fitx.copy()
        right_line.recent_xfitted = right_fitx.copy()
        left_line.rw_xfitted = left_fit_rw.copy()
        right_line.rw_xfitted = right_fit_rw.copy()
        left_line.fit = left_fit.copy()
        right_line.fit = right_fit.copy()
                           
    # Create an image to draw the lines on
    warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))
    
    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    
    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
    
    # Colors in the left and right lane regions
    color_warp[lefty, leftx] = [255, 0, 0]
    color_warp[righty, rightx] = [0, 0, 255]
    
    return color_warp, left_curverad, right_curverad, deviation
    
    
def process_image(image) : 
        
    #correcting distortion 
    dst = cv2.undistort(image, mtx, dist, None, mtx)

    #extracting lines
    lanes = extract_lanes(dst)
    
    #applying perspective transformation
    bird_eye = cv2.warpPerspective(lanes, M,(image.shape[1], image.shape[0]), flags = cv2.INTER_LINEAR)   
    
    #obtaining the image with the lanes
    color_warp, left_curverad, right_curverad, deviation = fit_polynomial(bird_eye)
    
    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (image.shape[1], image.shape[0])) 
    
    # Combine the result with the original image
    result = cv2.addWeighted(dst, 1, newwarp, 0.3, 0)
    
    # mean of the curvature
    curvature = np.mean ([np.abs(left_curverad), np.abs(right_curverad)])
    
    # direction left/right
    direction = ''
    if deviation > 0 : 
        direction = 'left'
    else :
        direction = 'right'
    
    # printing on top of the image
    cv2.putText(result,'Radius of Curvature = {:.1f}(m)'.format(curvature),(30,50), cv2.FONT_HERSHEY_SIMPLEX, 2,(255,255,255),2)
    cv2.putText(result,'Vehicle is {:.3f} {} of the center'.format(deviation, direction),(30,100), cv2.FONT_HERSHEY_SIMPLEX, 2,(255,255,255),2)
    
    # saving the image
    return result



In [36]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML

white_output = 'project_video_output.mp4'

#readign the first 5 seconds of the video
clip1 = VideoFileClip("project_video.mp4")#.subclip(20,30) 
white_clip = clip1.fl_image(process_image) 
%time white_clip.write_videofile(white_output, audio=False)

                                                              


                                                     
                                                           

t:  31%|███       | 151/485 [47:48<00:27, 12.37it/s, now=None]


t:  31%|███       | 151/485 [38:25<00:27, 12.00it/s, now=None]

                                                              


                                                     
                                                           

t:  31%|███       | 151/485 [47:48<00:27, 12.37it/s, now=None]


t:  31%|███       | 151/485 [38:25<00:27, 12.00it/s, now=None]

t:  31%|███       | 151/485 [37:07<00:31, 10.46it/s, now=None]



t:   0%|          | 0/1260 [00:00<?, ?it/s, now=None]

from memory 261 1052

281.5769732990147 384.2553980972159
all good
Moviepy - Building video project_video_output.mp4.
Moviepy - Writing video project_video_output.mp4

from memory 358 1024

271.76933441647725 384.2553980972159
all good






t:   0%|          | 2/1260 [00:00<01:37, 12.84it/s, now=None]



t:   0%|          | 3/1260 [00:00<02:01, 10.33it/s, now=None]



t:   0%|          | 4/1260 [00:00<02:20,  8.93it/s, now=None]

from memory 368 1024

266.4534309633791 381.00171642860846
all good
from memory 373 1021

267.7151771797743 384.1814548783311
all good






t:   0%|          | 5/1260 [00:00<02:19,  8.98it/s, now=None]

...

from memory 385 1111

255.7675505068981 416.67900051111496
all good
from memory 384 1056

256.90929569822964 411.71587597115695
all good
from memory 383 1051

253.68991578916535 407.64494536600137
all good






t: 100%|██████████| 1260/1260 [02:00<00:00, 10.55it/s, now=None]



                                                                

from memory 386 1047

256.3108526722044 403.83294457870716
all good
from memory 383 1043

256.9835071792553 401.6481554029606
all good


                                                              


                                                     
                                                           

t:  31%|███       | 151/485 [49:49<00:27, 12.37it/s, now=None]


t:  31%|███       | 151/485 [40:26<00:27, 12.00it/s, now=None]

                                                              


                                                     
                                                           

t:  31%|███       | 151/485 [49:49<00:27, 12.37it/s, now=None]


t:  31%|███       | 151/485 [40:26<00:27, 12.00it/s, now=None]

t:  31%|███       | 151/485 [39:08<00:31, 10.46it/s, now=None]

Moviepy - Done !
Moviepy - video ready project_video_output.mp4
CPU times: user 2min 32s, sys: 11.6 s, total: 2min 44s
Wall time: 2min 1s
